# Dimensionality Reduction
#### Roles and Buckets

In [ ]:
!pip install mxnet

In [1]:
# sagemaker libraries
import boto3
import sagemaker
from sagemaker import get_execution_role

# pandas and np
import numpy as np
import pandas as pd

import os

In [2]:
session = sagemaker.Session() # store the current SageMaker session

# get IAM role
role = get_execution_role()
print(role)

arn:aws:iam::305472246504:role/service-role/AmazonSageMaker-ExecutionRole-20210617T231874


In [3]:
# get default bucket
bucket = session.default_bucket()
print(bucket)
print()

sagemaker-us-east-1-305472246504



In [4]:
# should be the name of directory you created to save your features data
data_dir = 'saved_progress'

# set prefix, a descriptive name for a directory  
prefix = 'starbucks/pca-2021-06-18-05-52-03-430/output'

# upload all data to S3
input_data = session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
print(input_data)

s3://sagemaker-us-east-1-305472246504/starbucks/pca-2021-06-18-05-52-03-430/output


#### Define a PCA Model
* role: The IAM role, which was specified, above.
* train_instance_count: The number of training instances (typically, 1).
* train_instance_type: The type of SageMaker instance for training.
* num_components: An integer that defines the number of PCA components to produce.
* sagemaker_session: The session used to train on SageMaker.

In [ ]:
# define location to store model artifacts
prefix = 'starbucks'

output_path='s3://{}/{}/'.format(bucket_name, prefix)

print('Training artifacts will be uploaded to: {}'.format(output_path))

In [ ]:
data_scaled.shape[1]

In [ ]:
# define a PCA model
from sagemaker import PCA

data_scaled = pd.read_csv('saved_progress/scaled_dataset.csv')
data_scaled.drop(columns=['Unnamed: 0'], inplace=True)

# this is current features - 1
# you'll select only a portion of these to use, later
N_COMPONENTS = data_scaled.shape[1] - 1

pca_starbucks = PCA(role=role,
             instance_count=1,
             instance_type='ml.p2.xlarge',
             output_path=output_path, # specified, above
             num_components=N_COMPONENTS, 
             sagemaker_session=session)

#### Convert data into a RecordSet format¶
Next, prepare the data for a built-in model by converting the DataFrame to a numpy array of float values.

The record_set function in the SageMaker PCA model converts a numpy array into a RecordSet format that is the required format for the training input data. This is a requirement for all of SageMaker's built-in models. The use of this data type is one of the reasons that allows training of models within Amazon SageMaker to perform faster, especially for large datasets.

In [ ]:
# convert df to np array
train_data_np = data_scaled.values.astype('float32')

# convert to RecordSet format
formatted_train_data = pca_starbucks.record_set(train_data_np)

#### Train the model

In [ ]:
%%time

# train the PCA mode on the formatted data
pca_starbucks.fit(formatted_train_data)

In [ ]:
# Get the name of the training job, it's suggested that you copy-paste
# from the notebook or from a specific job in the AWS console
training_job_name='pca-2021-06-18-05-52-03-430'

# where the model is saved, by default
model_key = os.path.join(prefix, training_job_name, 'output/model.tar.gz')
print(model_key)

# download and unzip model
boto3.resource('s3').Bucket(bucket_name).download_file(model_key, 'model.tar.gz')

# unzipping as model_algo-1
os.system('tar -zxvf model.tar.gz')
os.system('unzip model_algo-1')

In [ ]:
import mxnet as mx

# loading the unzipped artifacts
pca_model_params = mx.ndarray.load('model_algo-1')

# what are the params
print(pca_model_params)

#### PCA Model Attributes
Three types of model attributes are contained within the PCA model.

**mean**: The mean that was subtracted from a component in order to center it.

**v**: The makeup of the principal components; (same as ‘components_’ in an sklearn PCA model).

**s**: The singular values of the components for the PCA transformation. This does not exactly give the % variance from the original feature space, but can give the % variance from the projected feature space.

We are only interested in v and s.

In [ ]:
# get selected params
s=pd.DataFrame(pca_model_params['s'].asnumpy())
v=pd.DataFrame(pca_model_params['v'].asnumpy())

#### Data Variance
Our current PCA model creates 33 principal components, but when we create new dimensionality-reduced training data, we'll only select a few, top n components to use. To decide how many top components to include, it's helpful to look at how much data variance the components capture.



When we select the top n components to use in a new data model, we'll typically want to include enough components to capture about 80-90% of the original data variance. In this project, we are looking at generalizing over a lot of data and we'll aim for about 85% coverage.

Note: The top principal components, with the largest s values, are actually at the end of the s DataFrame. Let's print out the s values for the top n, principal components.

In [ ]:
# looking at top 5 components
n_principal_components = 5

start_idx = N_COMPONENTS - n_principal_components  

# print a selection of s
print(s.iloc[start_idx:, :])

In [ ]:

def explained_variance(s, n_top_components):
    '''Calculates the approx. data variance that n_top_components captures.
       :param s: A dataframe of singular values for top components; 
           the top value is in the last row.
       :param n_top_components: An integer, the number of top components to use.
       :return: The expected data variance covered by the n_top_components.'''
    
    start_idx = N_COMPONENTS - n_top_components 
    # calculate approx variance
    exp_variance = np.square(s.iloc[start_idx:,:]).sum()/np.square(s).sum()
    
    return exp_variance[0]

In [ ]:
# test cell
n_top_components = 5 # select a value for the number of top components

# calculate the explained variance
exp_variance = explained_variance(s, n_top_components)
print('Explained variance: ', exp_variance)

#### Component Makeup
We can now examine the makeup of each PCA component based on **the weightings of the original features that are included in the component**. The following code shows the feature-level makeup of the first component.

Note that the components are again ordered from smallest to largest and so I am getting the correct rows by calling N_COMPONENTS-1 to get the top, 1, component.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 

def display_component(v, features_list, component_num, n_weights=10):
    
    # get index of component (last row - component_num)
    row_idx = N_COMPONENTS-component_num

    # get the list of weights from a row in v, dataframe
    v_1_row = v.iloc[:, row_idx]
    v_1 = np.squeeze(v_1_row.values)

    # match weights to features in counties_scaled dataframe, using list comporehension
    comps = pd.DataFrame(list(zip(v_1, features_list)), 
                         columns=['weights', 'features'])

    # we'll want to sort by the largest n_weights
    # weights can be neg/pos and we'll sort by magnitude
    comps['abs_weights']=comps['weights'].apply(lambda x: np.abs(x))
    sorted_weight_data = comps.sort_values('abs_weights', ascending=False).head(n_weights)

    # display using seaborn
    ax=plt.subplots(figsize=(10,6))
    ax=sns.barplot(data=sorted_weight_data, 
                   x="weights", 
                   y="features", 
                   palette="Blues_d")
    ax.set_title("PCA Component Makeup, Component #" + str(component_num))
    plt.show()

In [ ]:
# display makeup of first component
num = 5
display_component(v, data_scaled.columns.values, component_num=num, n_weights=10)

#### Deploying the PCA Model
We can now deploy this model and use it to make "predictions". Instead of seeing what happens with some test data, we'll actually want to pass our training data into the deployed endpoint to create principal components for each data point.

In [ ]:
%%time
# this takes a little while, around 8mins
pca_predictor = pca_starbucks.deploy(initial_instance_count=1, 
                              instance_type='ml.p2.xlarge')

In [ ]:
# pass np train data to the PCA model
train_pca = pca_predictor.predict(train_data_np)

In [ ]:
# check out the first item in the produced training features
data_idx = 0
print(train_pca[data_idx])

In [ ]:

def create_transformed_df(train_pca, data_scaled, n_top_components):
    ''' Return a dataframe of data points with component features. 
        The dataframe should be indexed by State-County and contain component values.
        :param train_pca: A list of pca training data, returned by a PCA model.
        :param counties_scaled: A dataframe of normalized, original features.
        :param n_top_components: An integer, the number of top components to use.
        :return: A dataframe, indexed by State-County, with n_top_component values as columns.        
     '''
    # create new dataframe to add data to
    data_transformed=pd.DataFrame()

    # for each of our new, transformed data points
    # append the component values to the dataframe
    for data in train_pca:
        # get component values for each data point
        components=data.label['projection'].float32_tensor.values
        data_transformed = data_transformed.append([list(components)])

    # index by county, just like counties_scaled
    data_transformed.index = data_scaled.index

    # keep only the top n components
    start_idx = N_COMPONENTS - n_top_components
    data_transformed = data_transformed.iloc[:,start_idx:]
    
    # reverse columns, component order     
    return data_transformed.iloc[:, ::-1]

In [ ]:
# specify top n
top_n = 5

# call your function and create a new dataframe
data_transformed = create_transformed_df(train_pca, data_scaled, n_top_components=top_n)

# add descriptive columns
PCA_list=['c_1', 'c_2', 'c_3', 'c_4', 'c_5']
data_transformed.columns=PCA_list 

# print result
data_transformed.head()

In [ ]:

# Save PCA dataset
data_transformed.to_csv('saved_progress/pca_scores.csv')

In [ ]:
# Delete predictor endpoint
session.delete_endpoint(pca_predictor.endpoint)

#### Metrics
**Eblow method**
Inertia measures how well a data set is clustered by K-Means. It is calculated by measuring the distance between each data point and its centroid, squaring this distance and summing these squares into a cluster.

A good model is one with low inertia AND a low number of clusters (K). However, this is a trade-off because as K increases, the inertia decreases.

To find the optimal K for a data set, use the Elbow method; find the point where the decrease in inertia begins to decrease.

In [ ]:
from sklearn.cluster import KMeans
import sklearn.metrics as metrics

scores = []

for k in range(1,11):
    km = KMeans(n_clusters = k,random_state=1)
    km = km.fit(data_transformed)
    scores.append(km.inertia_)

dfk = pd.DataFrame({'Cluster':range(1,11), 'Score':scores})
plt.figure(figsize=(8,5))
plt.plot(dfk['Cluster'], dfk['Score'], marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

#### Silhouette value
The silhouette value measures how similar a point is to its own cluster (cohesion) compared to other clusters (separation).

In [ ]:
for i in range(2,17):
    kmeans_labels=KMeans(n_clusters=i,random_state=1).fit_predict(data_transformed)
    print("Silhouette score for {} clusters k-means : {} ".format(i,metrics.silhouette_score(data_scaled,kmeans_labels, metric='euclidean').round(3)))

#### Davies Bouldin
Average similarity measure of each cluster to its most similar cluster, where similarity is the ratio of distances within the cluster to distances between clusters. The minimum score is zero, and lower values indicate better clustering.

In [ ]:
for i in range(2, 17):
    kmeans_labels=KMeans(n_clusters=i,random_state=1).fit_predict(data_transformed)
    print(f'Davies Bouldin Score {i}: {metrics.davies_bouldin_score(data_scaled,kmeans_labels).round(3)}')

In [ ]:
# define a KMeans estimator
from sagemaker import KMeans

NUM_CLUSTERS = 3

kmeans = KMeans(role=role,
                instance_count=1,
                instance_type='ml.p2.xlarge',
                output_path=output_path, # using the same output path as was defined, earlier              
                k=NUM_CLUSTERS)

#### Create formatted, k-means training data
Just as before, you should convert the data_transformed df into a numpy array and then into a RecordSet. This is the required format for passing training data into a KMeans model.

In [ ]:
# convert the transformed dataframe into record_set data
kmeans_train_data_np = data_transformed.values.astype('float32')
kmeans_formatted_data = kmeans.record_set(kmeans_train_data_np)

In [ ]:
%%time

# train kmeans
kmeans.fit(kmeans_formatted_data)

#### Deploy the k-means model

In [ ]:
%%time
# deploy the model to create a predictor
kmeans_predictor = kmeans.deploy(initial_instance_count=1, 
                                 instance_type='ml.p2.xlarge')

#### Pass in the training data and assign predicted cluster labels
After deploying the model, you can pass in the k-means training data, as a numpy array, and get resultant, predicted cluster labels for each data point.

In [ ]:
# get the predicted clusters for all the kmeans training data
cluster_info=kmeans_predictor.predict(kmeans_train_data_np)

#### Exploring the resultant clusters
The resulting predictions should give you information about the cluster that each data point belongs to.

In [ ]:
# print cluster info for first data point
data_idx = 0

print('Person is: ', data_transformed.index[data_idx])
print()
print(cluster_info[data_idx])

#### Visualize the distribution of data over clusters
Get the cluster labels for each of our data points (counties) and visualize the distribution of points over each cluster.

In [ ]:
# get all cluster labels
cluster_labels = [c.label['closest_cluster'].float32_tensor.values[0] for c in cluster_info]

In [ ]:
# count up the points in each cluster
cluster_df = pd.DataFrame(cluster_labels)[0].value_counts()

print(cluster_df)

In [ ]:
ax =plt.subplots(figsize=(6,3))
ax = plt.hist(cluster_labels, bins=8,  range=(-0.5, 2.5), color='blue', rwidth=0.5)

title="Histogram of Cluster Counts"
plt.title(title, fontsize=12)
plt.show()

#### Delete the Endpoint!
Now that you've deployed the k-means model and extracted the cluster labels for each data point, you no longer need the k-means endpoint.

In [ ]:
# Delete kmeans endpoint
session.delete_endpoint(kmeans_predictor.endpoint)

### Model Attributes & Explainability
Explaining the result of the modeling is an important step in making use of our analysis. By combining PCA and k-means, and the information contained in the model attributes within a SageMaker trained model, you can learn about a population and remark on some patterns you've found, based on the data.

In [ ]:
# Download and unzip the kmeans model file
kmeans_job_name = 'kmeans-2021-06-19-18-15-16-449'

model_key = os.path.join(prefix, kmeans_job_name, 'output/model.tar.gz')

# download the model file
boto3.resource('s3').Bucket(bucket_name).download_file(model_key, 'model.tar.gz')
os.system('tar -zxvf model.tar.gz')
os.system('unzip model_algo-1')

In [ ]:

# Get the trained kmeans params using mxnet
kmeans_model_params = mx.ndarray.load('model_algo-1')

print(kmeans_model_params)

In [ ]:
# Get all the centroids
cluster_centroids = pd.DataFrame(kmeans_model_params[0].asnumpy())
cluster_centroids.columns = data_transformed.columns

display(cluster_centroids)

In [ ]:
# Save clusteres
cluster_centroids.to_csv('saved_progress/cluster_centroids.csv')

#### Visualizing Centroids in Component Space

In [ ]:
# generate a heatmap in component space, using the seaborn library
plt.figure(figsize = (12,9))
ax = sns.heatmap(cluster_centroids.T, cmap = 'YlGnBu')
ax.set_xlabel("Cluster")
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
ax.set_title("Attribute Value by Centroid")
plt.show()

#### Natural Grouping

In [ ]:

# add a 'labels' column to the dataframe
data_transformed['labels']=list(map(int, cluster_labels))

# sort by cluster label 0-6
sorted_clients = data_transformed.sort_values('labels', ascending=True)
# view some pts in cluster 0
sorted_clients.head(20)

In [ ]:
cluster_centroids.to_csv('saved_progress/kmeans_clusters.csv')

In [ ]:
# Get all counties with label == 1
cluster = data_transformed[data_transformed['labels']==1]
cluster.head()

In [ ]:
!pip install hdbscan

In [ ]:
import hdbscan

In [ ]:
data = pd.read_csv('saved_progress/scaled_dataset.csv')
data.drop(columns=["Unnamed: 0" ], inplace=True)
data

In [ ]:
data = data.to_numpy()

In [ ]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=25, gen_min_span_tree=True)
clusterer.fit(data)


In [ ]:
clusterer.minimum_spanning_tree_.plot(edge_cmap='viridis', 
                                      edge_alpha=0.6, 
                                      node_size=80, 
                                      edge_linewidth=2)

In [ ]:
labels = pd.DataFrame(clusterer.labels_)
labels[0].unique()

In [18]:
!pip install hdbscan==0.8.27
!pip install --upgrade pip

In [19]:

# your import and estimator code, here
from sagemaker.sklearn.estimator import SKLearn

estimator = SKLearn(entry_point="train.py",
                    source_dir="source_sklearn",
                    role=role,
                    framework_version="0.20.0",
                    py_version="py3",
                    train_instance_count=1,
                    train_instance_type='ml.c4.xlarge')

train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [6]:
# !pip install -r "source_sklearn/requirements.txt"

In [20]:
%%time

# Train your estimator on S3 training data
estimator.fit({'train': input_data})

2021-06-22 05:41:13 Starting - Starting the training job...
2021-06-22 05:41:15 Starting - Launching requested ML instancesProfilerReport-1624340473: InProgress
...
2021-06-22 05:42:11 Starting - Preparing the instances for training.........
2021-06-22 05:43:41 Downloading - Downloading input data...
2021-06-22 05:44:09 Training - Downloading the training image..2021-06-22 05:44:24,739 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2021-06-22 05:44:24,741 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-06-22 05:44:24,751 sagemaker_sklearn_container.training INFO     Invoking user training script.

2021-06-22 05:44:29 Training - Training image download completed. Training in progress.2021-06-22 05:45:09,401 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-06-22 05:45:09,413 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-06-22 05:4

UnexpectedStatusException: Error for Training job sagemaker-scikit-learn-2021-06-22-05-41-13-397: Failed. Reason: AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_containers/_trainer.py", line 84, in train
    entrypoint()
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_sklearn_container/training.py", line 39, in main
    train(environment.Environment())
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_sklearn_container/training.py", line 35, in train
    runner_type=runner.ProcessRunnerType)
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/entry_point.py", line 100, in run
    wait, capture_error
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/process.py", line 164, in run
    cwd=environment.code_dir,
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/process.py", line 84, in check_error
    raise error_class(return_code=return_code, cm

In [9]:
%%time

# uncomment, if needed
# from sagemaker.pytorch import PyTorchModel


# deploy your model to create a predictor
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

ClientError: An error occurred (ValidationException) when calling the CreateModel operation: Could not find model data at s3://sagemaker-us-east-1-305472246504/sagemaker-scikit-learn-2021-06-22-04-48-20-397/output/model.tar.gz.